In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from openai import OpenAI

client = OpenAI()

JSON Mode <br>
https://platform.openai.com/docs/guides/structured-outputs#json-mode 

Cechy:
- w nowych modelach odchodzi się od używania JSON Mode (wyparte przez inne rozwiązania)
- NIE GWARANTUJE, że odpowiedź będzie zgodna ze schematem
- W prompcie KONIECZNIE trzeba zawrzeć słowo JSON - inaczej dostaniemy błąd `Error code: 400 - {'error': {'message': "'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object'."`

In [3]:
system_msg = """
You are a helpful assistant designed to translate given sentences by user to Spanish, Italian and Polish. Always respond in JSON. 
Response format should be like: {"spanish": ..., "italian": ..., "polish": ...}
"""

In [4]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": "Hi, how are you doing?"}
        ],
        response_format={"type": "json_object"}
    )

In [5]:
import json

json.loads(response.choices[0].message.content)

{'spanish': 'Hola, ¿cómo estás?',
 'italian': 'Ciao, come stai?',
 'polish': 'Cześć, jak się masz?'}

In [6]:
response.usage.model_dump()

{'completion_tokens': 40,
 'prompt_tokens': 66,
 'total_tokens': 106,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

JSON Schema (Pydantic)

In [7]:
from pydantic import BaseModel

class TransaltionSchema(BaseModel):
    spanish: str
    italian: str
    polish: str


In [8]:
response = client.beta.chat.completions.parse(  # this is beta for now!
        model="gpt-4o-mini",  # model need to be changed
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to translate given sentences by user to Spanish, Italian and Polish."},  # do not tell the model to return JSON
            {"role": "user", "content": "Hi, how are you doing?"}
        ],
        response_format=TransaltionSchema  # here is the difference in repsonse format
    )

In [9]:
json.loads(response.choices[0].message.content)

{'spanish': 'Hola, ¿cómo estás?',
 'italian': 'Ciao, come stai?',
 'polish': 'Cześć, jak się masz?'}

In [10]:
response.usage.model_dump()

{'completion_tokens': 34,
 'prompt_tokens': 104,
 'total_tokens': 138,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}